# Practicing with Langchain LLM Models  

<div style="background-color:rgba(205, 200, 225, 0.7); padding:7px 7px;">

**Notes:** 
- Make sure the notebook environment is set to `venv` (top right).

</div>

### Set Up

In [1]:
# imports 
from langchain.llms import OpenAI # for OpenAI models 
from langchain import HuggingFaceHub # for Huggingface models 
from langchain.prompts import PromptTemplate # for prompt templates 
from langchain.chains import LLMChain # for prompt templates 
from langchain.chains import SimpleSequentialChain # for simple sequential chains 
from langchain.chains import SequentialChain # for sequential chains 
from langchain.chat_models import ChatOpenAI # for ChatOpenAI 
from langchain.schema import HumanMessage, SystemMessage, AIMessage # for ChatOpenAI 
from langchain.prompts.chat import ChatPromptTemplate # for output parsers
from langchain.schema import BaseOutputParser # for output parsers
import os # for the keys 
import warnings
warnings.filterwarnings('ignore')

## **Creating LLM Models**

### OpenAI Models

<div style="background-color:rgba(205, 200, 225, 0.7); padding:7px 7px;">

**Links:**
- OpenAI API: [OpenAI](https://openai.com/blog/openai-api)
- Documentation about the OpenAI models: [documentation](https://platform.openai.com/docs/overview)

**Notes:**
- **`temperature`**: how creative we want our model to be
    - `=0`: the model is very safe and not "taking any bets"
    - `=1`: more creative and risky, may give wrong outputs
</div>

In [2]:
# add API key 
os.environ["OPEN_API_KEY"]="ENTER-YOUR-API-KEY-HERE"

In [3]:
# create the llm model 
llm = OpenAI(openai_api_key=os.environ["OPEN_API_KEY"],
             temperature=0.6)

In [4]:
# try a prediction 
text = "What is the capital of Canada?"
print(llm.predict(text))



The capital of Canada is Ottawa.


### Huggingface Models

<div style="background-color:rgba(205, 200, 225, 0.7); padding:7px 7px;">

**Notes:**
- **`repo_id`**: the repository of the model on Huggingface you want to use
    - e.g. [huggingface.co/google/flan-t5-large](https://huggingface.co/google/flan-t5-large)

</div>

In [5]:
# add API key 
os.environ["HUGGINGFACEHUB_API_TOKEN"]="ENTER-YOUR-API-KEY-HERE"

In [6]:
# create the llm model 
llm_huggingface = HuggingFaceHub(repo_id="google/flan-t5-large",
                                 model_kwargs={"temperature": 0, 
                                               "max_length": 64})

In [7]:
# try a prediction
text = "Can you tell me the capital of England?"
print(llm_huggingface.predict(text))

london


### Compare the two models 

<div style="background-color:rgba(154, 198, 149, 0.5); padding:10px 7px;">

It looks like the `OpenAI` model performs better. 

</div>

In [8]:
# OpenAI
llm.predict("Can you write me a poem about apples?")

"\n\nCrisp and round, a vibrant red\nIn orchards, they grow in a bed\nJuicy and sweet, a perfect treat\nThe apple, oh how it's hard to beat\n\nFrom humble seeds, they start to grow\nThrough rain and sun, they begin to show\nTheir shiny skin, a sight to behold\nIn baskets and bowls, they're often sold\n\nBut apples are more than just a snack\nThey have a history, you can't keep track\nFrom Snow White's bite to Newton's law\nThey've been a symbol, a metaphor\n\nIn autumn, they paint the trees\nA fiery display, a sight to please\nRipe for picking, they fall to the ground\nA harvest bounty, for all to be found\n\nIn pies and crumbles, they're a delight\nBaked with cinnamon, a scrumptious bite\nIn cider and juice, they quench our thirst\nA refreshing drink, for the best and worst\n\nBut let's not forget, their healthful side\nAn apple a day, keeps the doctor aside\nLoaded with vitamins, they boost our health\nA natural remedy, for all our wealth\n\nSo here's to the apple, a fruit so fine\nI

In [9]:
# huggingface 
llm_huggingface.predict("Can you write me a poem about apples?")

'apple i love apple i love apple i love apple i love apple i'

## **Using Prompt Templates and LLM Chains**


<div style="background-color:rgba(205, 200, 225, 0.7); padding:7px 7px;">

**Notes:** 
- We can use a prompt template if we want to execute the same type of query multiple times
- In order to use a template with our LLM, we must also use a "chain"

</div>

In [10]:
# create a prompt template 
prompt_template = PromptTemplate(input_variables=["country"],
                                 template="Tell me the capical of {country}")
prompt_template.format(country="Japan")

'Tell me the capical of Japan'

In [11]:
# THIS WILL NOT WORK (because we're not using chains)
#llm.predict(prompt_template=prompt_template, text=["Japan"])

In [12]:
# create an LLM chain to use our prompt template with our llm 
chain = LLMChain(llm=llm,
                 prompt=prompt_template)
print(chain.run("Japan"))



The capital of Japan is Tokyo.


## **Combining Multiple Chains Using Simple Sequential Chains**


<div style="background-color:rgba(205, 200, 225, 0.7); padding:7px 7px;">

**Notes:**
- The last `chain.run("Canada")` command did not print everything - to get the entire output we need to access the buffer 

</div>

In [13]:
# create one prompt template 
capital_template = PromptTemplate(input_variables=["country"],
                                template="Please tell me the capital of {country}")
# create a chain 
capital_chain = LLMChain(llm=llm, prompt=capital_template)

# create a second prompt template
visit_template = PromptTemplate(input_variables=["capital"],
                                template="Suggest to me some exciting places to visit in {capital}")
# create a second chain 
visit_chain = LLMChain(llm=llm, prompt=visit_template)

In [14]:
# create a simple sequential chain 
chain = SimpleSequentialChain(chains=[capital_chain, visit_chain])

# run the simple sequential chain, with country "Canada" 
print(chain.run("Canada"))



1. Parliament Hill - This iconic building is the seat of the Canadian government and offers guided tours, beautiful architecture, and stunning views of the city.

2. ByWard Market - This vibrant outdoor market is a must-visit for foodies, with a wide variety of fresh produce, meats, cheeses, and local specialties. You can also find unique boutiques, artisanal crafts, and street performers here.

3. National Gallery of Canada - Art lovers will enjoy exploring this impressive gallery, which houses a vast collection of Canadian and international works, including pieces by the Group of Seven and the famous Maman spider sculpture.

4. Rideau Canal - In the winter, this UNESCO World Heritage Site turns into the world's largest skating rink. In the summer, it's a popular spot for boating, cycling, and picnicking.

5. Canadian Museum of History - Located in nearby Gatineau, Quebec, this museum showcases the country's rich history and culture through interactive exhibits, artifacts, and speci

## **Sequential Chains**

In [15]:
# create one prompt template 
capital_template = PromptTemplate(input_variables=["country"],
                                template="Please tell me the capital of {country}")
# create a chain 
capital_chain = LLMChain(llm=llm, prompt=capital_template, 
                         output_key="capital") # NOTE: this output_key is added 

# create a second prompt template
visit_template = PromptTemplate(input_variables=["capital"],
                                template="Suggest to me some exciting places to visit in {capital}")
# create a second chain 
visit_chain = LLMChain(llm=llm, prompt=visit_template, 
                       output_key="places") # NOTE: this output_key is added 

In [16]:
# create a sequential chain 
chain = SequentialChain(chains=[capital_chain, visit_chain],
                        input_variables=["country"], # NOTE: added 
                        output_variables=["capital", "places"]) # NOTE: added 

In [17]:
# run the entire chain 
output = chain({"country": "Japan"}) # output will be a dictionary

In [18]:
print(output["country"])
print(output["capital"], "\n")
print(output["places"])

Japan


The capital of Japan is Tokyo. 

 Here are some exciting places to visit in Tokyo:

1. Shibuya Crossing: Known as the busiest intersection in the world, Shibuya Crossing is a must-visit for anyone visiting Tokyo. Experience the chaos and energy as thousands of people cross the intersection at the same time.

2. Tokyo Skytree: Standing at 634 meters, Tokyo Skytree is the tallest tower in Japan and offers breathtaking views of the city. You can also visit the observation deck and enjoy a meal at one of the restaurants.

3. Tsukiji Fish Market: This famous fish market is a must-visit for seafood lovers. You can watch the famous tuna auction, try fresh sushi, and explore the bustling market.

4. Meiji Shrine: This Shinto shrine is dedicated to Emperor Meiji and is a peaceful oasis in the middle of the bustling city. Take a stroll through the beautiful gardens and witness traditional Japanese rituals.

5. Akihabara: Known as the electronic district, Akihabara is a haven for anime, m

## **Chatmodels with ChatOpenAI**


In [19]:
# create a chat llm 
chatllm = ChatOpenAI(openai_api_key=os.environ["OPEN_API_KEY"], 
                     temperature=0.6, 
                     model="gpt-3.5-turbo")

In [20]:
# create the schema in the form of a list 
print(chatllm(
    [SystemMessage(content="You are a comedian AI assistant"),
     HumanMessage(content="Please provide some comedy punchlines on AI")]))

content='1. "Artificial intelligence is like my ex-boyfriend - smart, but just couldn\'t understand human emotions."\n2. "Why did the AI go to therapy? It had an identity crisis - it couldn\'t decide if it was a computer or a toaster!"\n3. "I asked Siri if she believes in love at first sight. She said, \'I\'m sorry, I can\'t answer that, but I can search the web for you.\' Thanks for nothing, Siri!"\n4. "Why did the AI refuse to go on a date? It said, \'I\'m sorry, I can\'t find a connection with you. Please try again later.\'"\n5. "I told my AI assistant that I\'m feeling lonely. It replied, \'Don\'t worry, I\'m here for you. Just like the last 100 times you asked me the same question.\'"\n6. "What do you call an AI that tells jokes? A pun-ning algorithm!"\n7. "I asked my AI assistant for some dating advice. It replied, \'Just be yourself!\' Well, that\'s easy for you to say, you\'re a computer!"\n8. "Why did the AI cross the road? To optimize its path-finding algorithm and minimize t

## **Prompt Template + LLM + Output Parsers** 

<div style="background-color:rgba(205, 200, 225, 0.7); padding:7px 7px;">

To modify an output from the LLM. As we saw in the previous section, the output of `chatllm` was kind of messy and hard to read.

</div>

In [21]:
# create a class that inherits "BaseOutputParser"
class Commaseparatedoutput(BaseOutputParser):

    # create an output parser 
    def parse(self, text:str):
        return text.strip().split(",")

In [22]:
# make a template for the AI 
template = "You are a helpful assistant. When the user gives any input, you should generate 5 words in a command separated list."

# make a template for the human 
human_template = "{text}"

# make a chat prompt 
chatprompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template)])

In [23]:
# the pipe "|" creates the chain 
chain = chatprompt|chatllm|Commaseparatedoutput()

In [24]:
# with the output parser
chain.invoke({"text":"intelligent"})

['smart', ' clever', ' knowledgeable', ' wise', ' brilliant']

In [25]:
# WITHOUT the output parser
chain_noparser = chatprompt|chatllm
chain_noparser.invoke({"text":"intelligent"})

AIMessage(content='smart, clever, brilliant, knowledgeable, quick-witted')